# Étape 3: Modélisation, Évaluation et Optimisation

## 1. Importation des bibliothèques et chargement des données

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Chargement des données
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/y_test.csv').values.ravel()

print('Données chargées.')

Données chargées.


## 2. Entraînement et Évaluation des Modèles de base

In [2]:
models = {
    'Régression Logistique': LogisticRegression(random_state=42, max_iter=1000),
    'Arbre de Décision': DecisionTreeClassifier(random_state=42),
    'Forêt Aléatoire': RandomForestClassifier(random_state=42)
}

results = {}

for name, model in models.items():
    print(f'Entraînement du modèle: {name}')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_proba)
    }

results_df = pd.DataFrame(results).T
print('
Évaluation terminée.')

SyntaxError: unterminated string literal (detected at line 24) (183337686.py, line 24)

## 3. Comparaison des performances

In [3]:
results_df

NameError: name 'results_df' is not defined

## 4. Visualisation des Courbes ROC

In [4]:
plt.figure(figsize=(10, 8))
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taux de Faux Positifs')
plt.ylabel('Taux de Vrais Positifs')
plt.title('Courbes ROC des modèles')
plt.legend()
plt.show()

NameError: name 'models' is not defined

<Figure size 1000x800 with 0 Axes>

## 5. Optimisation des hyperparamètres pour Random Forest

In [5]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')

grid_search.fit(X_train, y_train)

print(f'Meilleurs hyperparamètres: {grid_search.best_params_}')
best_rf = grid_search.best_estimator_

ModuleNotFoundError: No module named '_posixsubprocess'

## 6. Évaluation du modèle optimisé

In [6]:
y_pred_best = best_rf.predict(X_test)
y_proba_best = best_rf.predict_proba(X_test)[:, 1]

optimized_results = {
    'Accuracy': accuracy_score(y_test, y_pred_best),
    'Precision': precision_score(y_test, y_pred_best),
    'Recall': recall_score(y_test, y_pred_best),
    'F1-Score': f1_score(y_test, y_pred_best),
    'AUC-ROC': roc_auc_score(y_test, y_proba_best)
}

print('Performance du modèle Random Forest optimisé:')
for metric, value in optimized_results.items():
    print(f'{metric}: {value:.4f}')

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Churn', 'Churn'], yticklabels=['Non-Churn', 'Churn'])
plt.title('Matrice de confusion du modèle optimisé')
plt.ylabel('Vraie étiquette')
plt.xlabel('Étiquette prédite')
plt.show()

NameError: name 'best_rf' is not defined

## 7. Importance des caractéristiques

In [7]:
feature_importances = pd.DataFrame(best_rf.feature_importances_, index=X_train.columns, columns=['Importance']).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 10))
sns.barplot(x=feature_importances.Importance, y=feature_importances.index)
plt.title('Importance des caractéristiques - Random Forest optimisé')
plt.show()

NameError: name 'best_rf' is not defined

## 8. Conclusion
Le modèle de Forêt Aléatoire (Random Forest), après optimisation des hyperparamètres, offre les meilleures performances pour prédire le churn client, avec un score AUC-ROC élevé. L'analyse de l'importance des caractéristiques révèle que la durée du contrat (`Contract`), la durée d'abonnement (`tenure`) et les frais mensuels (`MonthlyCharges`) sont les principaux facteurs prédictifs du churn. Ces informations sont cruciales pour orienter les stratégies de rétention client.